## ThD Cases

In [ ]:
import os, sys, shutil, math
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import datetime

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.Cases as ThDCases
import shilofue.PlotCase as PlotCase
import shilofue.ThDSubduction0.PlotCase as ThDPlotCase
import shilofue.TwoDSubduction0.PlotCase as TwoDPlotCase
import shilofue.ThDSubduction0.VtkPp as ThDVtkPp
import shilofue.TwoDSubduction0.VtkPp as TwoDVtkPp
from shilofue.PlotCombine import PlotCombineExecute
from shilofue.ThDSubduction0.PlotVisit import PREPARE_RESULT_OPTIONS, VISIT_OPTIONS
import shilofue.PlotDepthAverage as PlotDepthAverage

sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities

if os.path.abspath("/home/lochy/ASPECT_PROJECT/HaMaGeoLib/") not in sys.path:
    sys.path.append(os.path.abspath("/home/lochy/ASPECT_PROJECT/HaMaGeoLib/"))

#### Utility functions

In [ ]:
def get_slab_dimensions_3(x, y, z, Ro, is_chunk):
    '''
    Derives the length along the three dimensions of a subducting slab.

    Inputs:
        x (float): x-coordinate of the slab point.
        y (float): y-coordinate of the slab point.
        z (float): z-coordinate of the slab point.
        Ro (float): Outer radius of the spherical domain.
        is_chunk (bool): Flag indicating whether the geometry is a spherical chunk.

    Returns:
        tuple: A tuple containing (r, w, l):
            - r (float): Radius or z-coordinate depending on whether the geometry is a chunk.
            - w (float): Width of the slab in the y-dimension, or converted width for chunk geometry.
            - l (float): Length of the slab in the x-dimension, or converted length for chunk geometry.
    
    Description:
        - For chunk geometries, converts Cartesian coordinates to spherical coordinates and calculates
          width and length using the outer radius Ro and spherical angles.
        - For non-chunk geometries, returns the z, x, and y coordinates directly as radius, length, and width.
    '''
    if is_chunk:
        # Convert Cartesian coordinates to spherical coordinates for chunk geometry
        r, th1, ph1 = Utilities.cart2sph(x, y, z)
        w = Ro * (np.pi / 2.0 - th1)  # Calculate width using the spherical angle th1
        l = Ro * ph1  # Calculate length using the spherical angle ph1
    else:
        # For non-chunk geometry, use Cartesian coordinates directly
        r = z
        l = x
        w = y
    
    return r, w, l


def GetSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots trench position for a single time step and calculates the slab dip angle.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval used to prepare snapshots for slab morphology analysis.
        kwargs (dict): Additional options:
            - silence (bool, default=False): Suppresses warnings if set to True.
            - w_query (float, default=0.0): Width value used for querying trench position.
            - pin_depth (float, default=100e3): Depth value for pinning.
            - crust_only (int, default=0): Flag to specify crustal composition to consider (0: both, 1: upper, 2: lower).

    Returns:
        tuple: Three numpy arrays containing time steps, slab depths, and dip angles.
    
    Description:
        - Initializes VTK options and extracts trench and slab tip positions over multiple time snapshots.
        - Interpolates data to derive the slab dip angle at each time snapshot.
        - Handles missing data files with warnings (unless silenced) and skips to the next snapshot.
    '''
    silence = kwargs.get("silence", False)
    w_query = kwargs.get("w_query", 0.0)
    pin_depth = kwargs.get("pin_depth", 100e3)
    crust_only = kwargs.get("crust_only", 0)

    # Initialize VTK options and interpret case settings
    Visit_Options = VISIT_OPTIONS(case_dir)
    Visit_Options.Interpret()
    trench_initial_position = Visit_Options.options['TRENCH_INITIAL']

    # Prepare snapshots for slab morphology with a given time interval
    available_pvtu_snapshots = Visit_Options.get_snaps_for_slab_morphology(time_interval=time_interval_for_slab_morphology)
    n_snapshots = len(available_pvtu_snapshots)

    is_chunk = (Visit_Options.options['GEOMETRY'] == "chunk")
    Ro = Visit_Options.options['OUTER_RADIUS']

    # Lists to store results
    dips = []
    depths = []
    ts = []

    # Iterate through each snapshot to calculate dip angles
    for n in range(n_snapshots - 1):
        vtu_snapshot = available_pvtu_snapshots[n]

        # Construct file names based on crust_only flag
        if crust_only == 1:
            filename0 = "trench_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_%05d.txt" % vtu_snapshot
            filename2 = "trench_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)
        elif crust_only == 2:
            filename0 = "trench_l_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_l_%05d.txt" % vtu_snapshot
            filename2 = "trench_l_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)
        else:
            filename0 = "trench_lu_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_lu_%05d.txt" % vtu_snapshot
            filename2 = "trench_lu_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)

        # Construct file paths
        filein0 = os.path.join(case_dir, "vtk_outputs", filename0)
        filein1 = os.path.join(case_dir, "vtk_outputs", filename1)
        filein2 = os.path.join(case_dir, "vtk_outputs", filename2)

        # Check if files exist, issue warnings if not found
        if not os.path.isfile(filein0):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein0} is not found', UserWarning)
            continue
        if not os.path.isfile(filein1):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein1} is not found', UserWarning)
            continue

        # Get time and step information for the current snapshot
        _time, step = Visit_Options.get_time_and_step_by_snapshot(vtu_snapshot)

        # Load trench position data from file
        data0 = np.loadtxt(filein0)
        xs0, ys0, zs0 = data0[:, 0], data0[:, 1], data0[:, 2]
        rs0, ws0, ls0 = get_slab_dimensions_3(xs0, ys0, zs0, Ro, is_chunk)
        l_tr0 = np.interp(w_query, ws0, ls0)
        r_tr0 = np.interp(w_query, ws0, rs0)

        # Load slab tip depth data from file
        data1 = np.loadtxt(filein1)
        rs1 = data1[:, 2]
        depth = float(Visit_Options.options["OUTER_RADIUS"]) - np.min(rs1)
        depths.append(depth)
        ts.append(_time)

        # Load data for the curve at a specific depth
        data2 = np.loadtxt(filein2)
        xs2, ys2, zs2 = data2[:, 0], data2[:, 1], data2[:, 2]
        rs2, ws2, ls2 = get_slab_dimensions_3(xs2, ys2, zs2, Ro, is_chunk)
        l_tr2 = np.interp(w_query, ws2, ls2)
        r_tr2 = np.interp(w_query, ws2, rs2)

        # Calculate the dip angle and append to the list
        dip = -math.atan((r_tr0 - r_tr2) / (l_tr0 - l_tr2))
        print(f"vtu_snapshot = {vtu_snapshot}, l_tr0 = {l_tr0:.4e}, r_tr0 = {r_tr0:.4e}, l_tr2 = {l_tr2:.4e}, r_tr2 = {r_tr2:.4e}, dip = {dip:.4e}")  # Debug output
        dips.append(dip)

    # Convert lists to numpy arrays
    dips = np.array(dips)
    depths = np.array(depths)
    ts = np.array(ts)

    return ts, depths, dips


def PlotSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots the slab dip angle and trench position for a single time step.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval for preparing snapshots of slab morphology.
        kwargs (dict): Additional plotting options:
            - color (str or None): Color of the plot lines.
            - label (str or None): Label for the plot lines.
            - w_query (float, default=0.0): Query width value in y-dimension for plotting.
            - axis_twinx (matplotlib axis object or None): Axis object for plotting depth on a secondary y-axis.
            - axis (matplotlib axis object, required): Axis object for plotting the dip angle.
    
    Returns:
        matplotlib axis object: The axis used for plotting the dip angle.

    Description:
        - Retrieves time, depths, and dip angles using `GetSlabDipAngle`.
        - Uses a univariate spline to smooth the dip angle data for visualization.
        - Plots the original and splined dip angles over time.
        - If `w_query` is near zero, also plots the depth at the center on the secondary y-axis.
    '''

    _color = kwargs.get("color", None)
    _label = kwargs.get("label", None)
    w_query = kwargs.get("w_query", 0.0)
    ax_twinx = kwargs.get("axis_twinx", None)

    # Initiate the plot on the provided axis
    ax = kwargs.get('axis', None)
    if ax is None:
        raise ValueError("Axis object must be provided for plotting.")

    # Retrieve time, depths, and dip angles
    ts, depths, dips = GetSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs)
    # Plot the dip angles, converting from radians to degrees
    ax.plot(ts / 1e6, dips * 180.0 / np.pi, label="y = %.2f km" % (w_query / 1e3), color=_color)

    # Apply a univariate spline to smooth the dip angles
    # size of data must be bigger than 3 to make a spline
    assert(ts.size > 3)
    spline = UnivariateSpline(ts / 1e6, dips, s=0)  # s=0 means interpolation without smoothing
    ts_new = np.linspace(ts.min(), ts.max(), 1000)
    dips_splined = spline(ts_new / 1e6)

    # Plot the splined dip angles, converting from radians to degrees
    ax.plot(ts_new / 1e6, dips_splined * 180.0 / np.pi, "-.", label="y = %.2f km (splined)" % (w_query / 1e3), color=_color)

    # If w_query is near zero, plot the depth at the center on the secondary y-axis
    if w_query < 1e-6 and ax_twinx is not None:
        ax_twinx.plot(ts / 1e6, depths / 1e3, "--", color=_color)  # Plot depth in km

    return ax



def PlotTrenchDifferences(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots the differences in trench position and the slab tip depth over time.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval for preparing snapshots of slab morphology.
        kwargs (dict): Additional plotting options:
            - color (str or None): Color of the plot lines.
            - label (str or None): Label for the plot lines.
            - silence (bool, default=False): Suppresses warnings if set to True.
            - w_query (float, default=0.0): Width value used for querying trench position.
            - axis_twinx (matplotlib axis object or None): Axis object for plotting depth on a secondary y-axis.
            - axis (matplotlib axis object, required): Axis object for plotting trench differences.
    
    Returns:
        matplotlib axis object: The axis used for plotting the trench differences.

    Description:
        - Initializes VTK options and retrieves snapshots based on the specified time interval.
        - Loads trench position and slab tip depth data for each snapshot and interpolates the trench position.
        - Plots the difference in trench position from the initial value and, if applicable, plots the slab depth.
    '''
    _color = kwargs.get("color", None)
    _label = kwargs.get("label", None)
    silence = kwargs.get("silence", False)
    w_query = kwargs.get("w_query", 0.0)
    ax_twinx = kwargs.get("axis_twinx", None)

    # Initialize VTK options and interpret case settings
    Visit_Options = VISIT_OPTIONS(case_dir)
    Visit_Options.Interpret()
    trench_initial_position = Visit_Options.options['TRENCH_INITIAL']

    # Prepare snapshots for slab morphology with the given time interval
    available_pvtu_snapshots = Visit_Options.get_snaps_for_slab_morphology(time_interval=time_interval_for_slab_morphology)
    n_snapshots = len(available_pvtu_snapshots)

    is_chunk = (Visit_Options.options['GEOMETRY'] == "chunk")
    Ro = Visit_Options.options['OUTER_RADIUS']

    # Initialize the plot on the provided axis
    ax = kwargs.get('axis', None)
    if ax is None:
        raise ValueError("Axis object must be provided for plotting.")

    # Lists to store results
    dl_trs = []
    depths = []
    ts = []

    # Iterate through each snapshot to calculate trench differences
    for n in range(n_snapshots - 1):
        vtu_snapshot = available_pvtu_snapshots[n]

        # Construct file paths for trench position and slab tip depth
        filein0 = os.path.join(case_dir, "vtk_outputs", "trench_%05d.txt" % vtu_snapshot)
        filein1 = os.path.join(case_dir, "vtk_outputs", "center_profile_%05d.txt" % vtu_snapshot)

        # Check if files exist, issue warnings if not found
        if not os.path.isfile(filein0):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein0} is not found', UserWarning)
            continue
        if not os.path.isfile(filein1):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein1} is not found', UserWarning)
            continue

        # Get time and step information for the current snapshot
        _time, step = Visit_Options.get_time_and_step_by_snapshot(vtu_snapshot)

        # Load trench position data from file
        data0 = np.loadtxt(filein0)
        xs0, ys0, zs0 = data0[:, 0], data0[:, 1], data0[:, 2]
        rs0, ws0, ls0 = get_slab_dimensions_3(xs0, ys0, zs0, Ro, is_chunk)
        l_tr0 = np.interp(w_query, ws0, ls0)
        dl_trs.append(l_tr0)

        # Load slab tip depth data from file
        data1 = np.loadtxt(filein1)
        xs1, ys1, zs1 = data1[:, 0], data1[:, 1], data1[:, 2]
        rs1, ws1, ls1 = get_slab_dimensions_3(xs1, ys1, zs1, Ro, is_chunk)
        depth = Ro - np.min(rs1)
        depths.append(depth)
        ts.append(_time)

    # Convert lists to numpy arrays
    dl_trs = np.array(dl_trs)
    depths = np.array(depths)
    ts = np.array(ts)

    # Plot the differences in trench position, subtracting the initial value
    ax.plot(ts / 1e6, (dl_trs - dl_trs[0]) / 1e3, label="y = %.2f km" % (w_query / 1e3), color=_color)

    # If w_query is near zero, plot the depth at the center on the secondary y-axis
    if w_query < 1e-6 and ax_twinx is not None:
        ax_twinx.plot(ts / 1e6, depths / 1e3, "--", color=_color)  # Plot depth in km

    return ax, ax_twinx


#### System path

The path to the project directory

    local_TwoDSubduction_dir

The path to the project directory on server

    remote_TwoDSubduction_dir

In [ ]:
local_TwoDSubduction_dir = "/mnt/lochz/ASPECT_DATA/TwoDSubduction"
local_ThDSubduction_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction"
remote_ThDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/ThDSubduction"
assert(os.path.isdir(local_ThDSubduction_dir))

# check case name and local directory
case_name_2d = None # set an None initial value to determine whether the 3d case is connected to a 2d case
local_dir_2d = None

# py_temp file
py_temp_dir = os.path.join(ASPECT_LAB_DIR, "py_temp_files")
os.makedirs(py_temp_dir, exist_ok=True) # Ensure the directory exists

today_date = datetime.datetime.today().strftime("%Y-%m-%d") # Get today's date in YYYY-MM-DD format
py_temp_file = os.path.join(py_temp_dir, f"py_temp_{today_date}.sh")

if not os.path.exists(py_temp_file):
    bash_header = """#!/bin/bash
# =====================================================
# Script: py_temp.sh
# Generated on: {date}
# Description: Temporary Bash script created by Python
# =====================================================

""".format(date=today_date)
    with open(py_temp_file, "w") as f:
        f.write(bash_header)

print(f"File ensured at: {py_temp_file}")



#### case name

case name (relative path to local_TwoDSubduction_dir)

    case_name

In [ ]:
# case_name = "EBA_2d_consistent_7/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_1/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_3/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4_cd150"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology_old_density"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_density"
# case_name = "EBA_2d_consistent_8_5/eba3d_width61_c22_AR4_fix_density"

# Cases without a Peierls creep
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"

# Cases with the Peierls creep, a deep box
# case_name = "EBA_2d_consistent_8_6/eba3d_width51_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width51_sc22"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width61_sc22"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR3"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"; case_name_2d = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"
# case_name ="EBA_2d_consistent_8_6/eba3d_width61_c22_AR5"
# case_name = "EBA_2d_consistent_8_8/eba3d_width80_bw8000_sw2000_c22_AR4_yd300.0"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss300.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22_nlht"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_10/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22_ss100.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width140_c22_AR4"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h2000"
# case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw2000_sw500_c22_AR4" ; 
# case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw8000_sw2000_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"
# case_name = "EBA_2d_consistent_8_8/eba3d_width51_bw4000_sw1000_c22_AR4_yd300.0"; case_name_2d = "EBA_CDPT_3dconsistent_10/eba_cdpt_coh300_SA80.0_OA40.0_width51_sc22_ss300.0"
# case_name = "EBA_2d_consistent_8_8/eba3d_width80_bw2000_sw500_c22_AR4_yd300.0"; 

# Cases with the Peierls creep, a shallow box
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA40.0_OA20.0_width80_h1000_ss300.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss300.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4_yd100"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss100.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4_yd300"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss300.0"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h1000"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss1000000.0"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h2000"; case_name_2d = None

# case_name = "chunk_test/chunk_initial0"
# case_name = "chunk_test/chunk_initial1"
# case_name = "chunk_test/chunk_initial2"
# case_name = "chunk_test/chunk_initial3"
# case_name = "chunk_test/chunk_initial4"
# case_name = "chunk_test/chunk_initial5"
# case_name = "chunk_test/chunk_initial6"
# case_name = "chunk_test/chunk_initial7"
# case_name = "chunk_test/chunk_initial8"
# case_name = "chunk_test/chunk_initial8_T"
# case_name = "chunk_test/chunk_initial9"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width61_ss1000000.0"
# case_name = "chunk_test/chunk_initial9_re_test_1"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width61_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width51_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width140_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA40.0_OA20.0_width80_ss1000000.0"
# case_name = "chunk_test/chunk_initial9_re_test"
# case_name = "chunk_geometry1/eba3d_width80_bw4000_sw1000_yd500.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss500.0"
# case_name = "chunk_geometry1/eba3d_width80_bw4000_sw1000_yd100.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss100.0"
case_name = "chunk_geometry1/eba3d_width80_bw8000_sw2000_yd500.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss500.0"

local_dir = None; local_dir_2d = None
if case_name is not None:
    local_dir = os.path.join(local_ThDSubduction_dir, case_name)
    assert(os.path.isdir(local_dir))
if case_name_2d is not None:
    local_dir_2d = os.path.join(local_TwoDSubduction_dir, case_name_2d)
    assert(os.path.isdir(local_dir_2d))

#### Plot Linear Results and Prepare the Scripts for Visualization

The next block will generate plots of runtime, solver statistics, etc., and will compose a script to run in ParaView or VisIt.

- **time_range**: range of time to plot; when set to `None`, the full range is covered
- **time_interval**: interval of time to plot; when set to `None`, every step is covered

##### Run the Script in ParaView

Run the following command with the script generated in the last block:

    paraview --script slab.py

##### Options in the Script
    
Options available in the script to tweak the behavior:

- `RUN_FULL_SCRIPT=False`
- `CROSS_SECTION_DEPTH=False`
- `PLOT_ISOVOLUME_WITH_STREAMLINE=False`
- `PLOT_Y_SLICES=False`

In [ ]:
# todo_3d_visual
from hamageolib.research.haoyuan_2d_subduction.legacy_tools import PlotCaseRunTwoD, PlotCaseRunThD
from hamageolib.research.haoyuan_2d_subduction.post_process import CASE_OPTIONS as CASE_OPTIONS_TWOD
from hamageolib.research.haoyuan_2d_subduction.workflow_scripts  import run_2d_subduction_visualization

time_range = None
time_interval = None
# turn on plot_axis if I want to save a complete result
# turn off if I want to prepare for figures in a paper
plot_axis = False
graphical_steps = [11]; slices=None # specify steps
# step = "auto"; slices=3  # auto-figure out the steps, take the numebr of slices
max_velocity = -1.0  # rescale the color for velocity
rotation_plus = 0.47 # rotation of the frame along the lon when making plot

# the old paraview-excluded workflow
# ofile_list = ["slab.py"]; require_base=True
# the new pyvista+paraview workflow
ofile_list = ["slab1.py"]; require_base=True

if local_dir is not None:
    # 3d cases
    PlotCase.PlotCaseRun(local_dir, time_range=time_range, run_visual=False,\
            time_interval=time_interval, visualization="paraview", last_step=1)
    plt.close() # plot won't show up below
    
    Visit_Options = PlotCaseRunThD(local_dir, time_range=time_range, run_visual=False,\
            time_interval=time_interval, visualization="paraview", step=graphical_steps, plot_axis=plot_axis, max_velocity=max_velocity,\
                    rotation_plus=rotation_plus, ofile_list=ofile_list, require_base=require_base)
    plt.close() # plot won't show up below

if local_dir_2d is not None:
    # 2d cases
    CaseOptionsTwoD = CASE_OPTIONS_TWOD(local_dir)

    plot_types = ["upper_mantle"]

    config = {
    "RESULT_DIR": RESULT_DIR,                   # directory to write output .txt
    "py_temp_file": py_temp_file,          # where to write pvpython script
    "PlotCaseRun_base": PlotCase.PlotCaseRun,                               # your PlotCase module
    "PlotCaseRun_project": PlotCaseRunTwoD,                       # your TwoDPlotCase module

    # ---
    # Visualization and plotting options
    # True: save a complete result
    # False: prepare for figures in a paper
    # ---
    "plot_axis": False,
    "graphical_steps": graphical_steps,
    "slices": None,
    "max_velocity": -1.0,
    "plot_types": plot_types,
    "rotation_plus": rotation_plus,
    "additional_fields": [],
    "CaseOptions": CaseOptionsTwoD
    # todo_velo
    }       
    Visit_Options_2d = run_2d_subduction_visualization(local_dir_2d, config)
    
#     PlotCase.PlotCaseRun(local_dir_2d, time_range=None, run_visual=False,\
#             time_interval=None, visualization="paraview", step=step)
#     plt.close() # plot won't show up below
    
#     Visit_Options_2d = TwoDPlotCase.PlotCaseRun(local_dir_2d, time_range=None, run_visual=False,\
#             time_interval=None, visualization="paraview", step=step, plot_axis=plot_axis, max_velocity=max_velocity, plot_types=plot_types, 
#             slices=slices)
#     plt.close() # plot won't show up below

### 3D plot options

#### Camera

    renderView1.InteractionMode = '2D'
    renderView1.CameraPosition = [5284226.190518921, 3146806.4170835665, 14061897.922485484]
    renderView1.CameraFocalPoint = [5284226.190518921, 3146806.4170835665, 0.0]
    renderView1.CameraParallelScale = 1697847.5422377351

#### pinpoint: following block

In [ ]:
print(Visit_Options.options['GEOMETRY'])

In [ ]:
# in case of the chunk geometry, query the cartesian coordinates of a spherical point
# this is useful to interact with Paraview slices

if Visit_Options.options['GEOMETRY'] == "chunk":

    depth = 700e3
    lon = 45.0 * np.pi / 180.0

    r = Visit_Options_2d.options['OUTER_RADIUS'] - depth
    rotation_angle_lon = Visit_Options_2d.options['ROTATION_ANGLE']
    # lon = (20.0 + rotation_angle_lon) * np.pi / 180.0  # plus the rotation angle if finding point on the tranformed piece

    trench_edge_lat = Visit_Options.options["TRENCH_EDGE_LAT_FULL"] * 0.75
    trench_edge_lat_rad = trench_edge_lat * np.pi / 180.0

    # x, y, z = Utilities.ggr2cart(trench_edge_lat_rad,lon,r) # query a point near slab edge
    x, y, z = Utilities.ggr2cart(0.0,lon,r) # query a point near slab edge
    # x, y, z = Utilities.ggr2cart(lat,lon,r) # query a point
    print("(x, y, z) = (%.3f, %.3f, %.3f)" % (x, y, z))
    print("Rotation angle along longitude: %.4f" % rotation_angle_lon)

else:
    print("Geometry is %s, not chunk, skip." % Visit_Options.options['GEOMETRY'])

### Plot depth-average results

In [ ]:
plot_time = 0
# plot_time = 0.2e6

depth_average_path = os.path.join(local_dir, 'output', 'depth_average.txt')
assert(os.path.isfile(depth_average_path))

fig_path_base = os.path.join(local_dir, 'img', 'DepthAverage.pdf')
PlotDepthAverage.PlotDaFigure(depth_average_path, fig_path_base, time=plot_time)

#### Automazed workflow to finalize visualization

In [ ]:
finalize_visual = True

if finalize_visual:

    from IPython.display import Image, display
    from hamageolib.research.haoyuan_2d_subduction.workflow_scripts import finalize_visualization_2d_12172024

    _time = 1.1e6
    
    # file types
    file_name = "viscosity"

    if file_name in ["viscosity", "T", "density", "metastable"]:
        frame_png_file_with_ticks = "/home/lochy/Documents/papers/documented_files/TwoDSubduction/upper_mantle_frame/upper_mantle_frame_12172024_trans-01.png"
    
        output_image_file = finalize_visualization_2d_12172024(local_dir, file_name, _time, frame_png_file_with_ticks, add_time=False)

### Visualization, pt 2: produce 2d slices

The reason for this operation is the big cases.
These cases cannot be loadly entirely due to memory shortage.
Thus the solution is to produce some slices

Note on the trade off between spacing and the split_perturbation parameters:
    
The spacing parameter tends to divide the domain into multiple spaces and accelerate
the process of interpolation, but make it harder to find the cell for a query point.
The problem is caused by the location of the cell center. When the cell is big, the cell center
might be far from a point within the cell, and one cell could be splited into different pieces of spacing.
This tackled by a large number of split perturbation, which will tell the interpolation algorithm to look into mulitple pices of spacing
rather than one and increases the possibility to locate the query point in a cell.
In application, first start with smaller spacing numbers. If the interpolation is slow, increase this number.

In [ ]:
vtu_snapshot = 25 + 4 # 4: usually the number of global refinement level

# print("python -m shilofue.ThDSubduction0.VtkPp slice_3d_geometry_beta -i %s -vss %d -sdl 5 -gs 100 100 100 -sr 800 300" % (local_dir, vtu_snapshot))

print("python -m shilofue.ThDSubduction0.VtkPp slice_3d_geometry_alpha -i %s -vss %s" % (local_dir, vtu_snapshot))

### Plot Slab Morphology

First analyze slab morphology (first cell).
Then plot slab morphology (second cell).

Interval to analyze.
0.5e6 is tested to give the best results in terms of capturing the trending and smearing out minor spikes.

    time_interval

In [ ]:
# the basic options to configure the slab envelop
slab_envelop_interval_w = 20e3  # Interval along x axis to sort out the trench locations
slab_envelop_interval_d = 20e3  # Interval along z axis to sort out the trench locations
slab_shallow_cutoff = 40e3  # Minimum depth along z axis to sort out the trench locations

In [ ]:
# generate the command for the slab envelop and run in terminal
crust_only = 1  # If we only use the crustal composition to sort out the trench locations
time_interval = 1e6 # The interval for trench position and velocities

print("python -m shilofue.ThDSubduction0.VtkPp morph_case_parallel -i %s -ti %.4e --slab_envelop_interval_w %.4e --slab_envelop_interval_d %.4e -ssc %.4e -co %d"\
        % (local_dir, time_interval, slab_envelop_interval_w, slab_envelop_interval_d, slab_shallow_cutoff, crust_only))

In [ ]:
# # plot the ternch velocity
# time_interval = 1e6

# SlabPlot = ThDVtkPp.SLABPLOT('slab')
# SlabPlot.PlotTrenchPosition(local_dir, time_interval=time_interval)
# # SlabPlot.PlotMorph(local_dir, save_pdf=True)

# plt.close()

### Analyze result from the 2d case

#### Plot trench migration with the 2d case

Here I combined the 3d trench migration at the center / edge to the 2d trench migration.
These results are plot in a single plot.

In [ ]:
# regenerate the 2d results if needed
# THis will first eliminate the old "slab_morph.txt" by option and then run the generation scripts below in series

# remove old results
remove_old_results = False
if remove_old_results:
    old_slab_morph_path = os.path.join(local_dir_2d, "vtk_outputs", "slab_morph.txt")
    if os.path.isfile(old_slab_morph_path):
        os.remove(old_slab_morph_path)

print("python command:")
print("python -m shilofue.TwoDSubduction0.VtkPp morph_case_parallel -i %s -ti %.4e" % (local_dir_2d, time_interval))

#### Get t when slab is at a query depth

In [ ]:
IndexByValue = lambda array_1d, val: np.argmin(abs(array_1d - val))

slab_morph_path_2d = os.path.join(local_dir_2d, "vtk_outputs", "slab_morph_t1.00e+05.txt")
assert(os.path.isfile(slab_morph_path_2d))

data = np.loadtxt(slab_morph_path_2d)
steps = data[:, 1]
times = data[:, 2]
trenches = data[:, 3]
slab_depths = data[:, 4]

# time of slab tip reaching 660 km and the index in the list
query_depth = 1000e3 # (for reproducing ribe's figure 300, 450, 600, 670 km), other common values: 660, 1000 km
sfunc = interp1d(slab_depths, times, assume_sorted=True)
t_query = sfunc(query_depth)
i_query = IndexByValue(times, t_query)
step_query = steps[i_query]
print("t%.0f = %.1f Ma, step = %d" % (query_depth/1e3, t_query/1e6, step_query))


#### Dip angle at the query depth

1. Slab tip dip angles at certain depth. Interact with the following parameters
    
    query_depth = 600e3
    
    dip_angle_depth_lookup = 540e3
    
    dip_angle_depth_lookup_interval = 60e3

The "query_depth" specifies the time step as when the slab reaches this depth. The "dip_angle_depth_lookup" specifies the depth to lookup the slab dip angle. While the "dip_angle_depth_lookup_interval" specifies the interval of depth to look up.

In [ ]:
# query for the slab dip angle sightly shallower than 660 km
query_depth = 600e3
dip_angle_depth_lookup = 540e3
dip_angle_depth_lookup_interval = 60e3

dip660 = TwoDVtkPp.GetSlabDipAt660(local_dir_2d, dip_angle_depth_lookup=dip_angle_depth_lookup, dip_angle_depth_lookup_interval=dip_angle_depth_lookup_interval, query_depth=query_depth)

print("dip660: %.4f (%.4f degree)" % (dip660, dip660/np.pi*180.0))

### Analyze result from the 3d case

#### Time step analyze

1. Specific time step for the slab to reach certain depth (e.g. 660 km, 800 km, 1000 km).

In [ ]:
IndexByValue = lambda array_1d, val: np.argmin(abs(array_1d - val))
yr = 365 * 24 * 3600.0
radius = 6371e3

Visit_Options = VISIT_OPTIONS(local_dir)
Visit_Options.Interpret() 

# get the t660 for the 3d case
time_interval_for_slab_morphology = 1e6
results = ThDVtkPp.CenterProfileAnalyze(local_dir, time_interval_for_slab_morphology)
t660, step660 = results['t660'], results['step660']
t700, step700 = results['t700'], results['step700']
t800, step800 = results['t800'], results['step800']
t1000, step1000 = results['t1000'], results['step1000']
print("t660: ", t660)
print("t700: ", t700)
print("t800: ", t800)
print("t1000: ", t1000)

# figure out the snapshot to analyze 
available_pvtu_times, available_pvtu_snapshots = Visit_Options.get_snaps_for_slab_morphology_outputs(time_interval=0.1e6)
print("available_pvtu_snapshots: ", available_pvtu_snapshots)
id = IndexByValue(available_pvtu_times, t660)
vtu_snapshot = available_pvtu_snapshots[id]
print("vtu_snapshot: ", vtu_snapshot)

#### Combine the plot of 2d and 3d slab morphologies

In [ ]:
plt.style.use('publication_2d_morph')

from shilofue.TwoDSubduction0.VtkPp import SLABPLOT as SLABPLOT2D
import warnings
from scipy.interpolate import interp1d

IndexByValue = lambda array_1d, val: np.argmin(abs(array_1d - val))

# connect the previous plot with a plot of the 2d case

fig = plt.figure(tight_layout=True, figsize=(20, 20))
gs = gridspec.GridSpec(2,1)
ax = fig.add_subplot(gs[0, 0])

############
# start the first plot: tip and trench position
ax_twinx = ax.twinx()
colors = ["tab:blue", "tab:orange"]
# First plot the 3d trench position
Visit_Options = VISIT_OPTIONS(local_dir)
Visit_Options.Interpret()
trench_edge_y = float(Visit_Options.options['TRENCH_EDGE_Y_FULL'])
w_queries = [0.0, trench_edge_y * 0.75]
time_interval_for_slab_morphology = 1e6
for i in range(len(w_queries)):
    w_query = w_queries[i]
    _color = colors[i]
    ax, ax_twinx = PlotTrenchDifferences(local_dir, time_interval_for_slab_morphology, axis=ax, w_query=w_query, silence=False, axis_twinx=ax_twinx, color=_color)

# get the t660 for the 3d case
results = ThDVtkPp.CenterProfileAnalyze(local_dir, time_interval_for_slab_morphology)
t660, step660 = results['t660'], results['step660']
print("3d case: t660 = %.4e Ma, step660 = %d" % (t660/1e6, step660))

if local_dir_2d is not None:

    # Then plot the 2d trench position
    SlabPlot2d = SLABPLOT2D('slab')
    TwoDVtkPp.PlotTrenchDifferences2dInter1Ma(SlabPlot2d, local_dir_2d, axis=ax, axis_twinx=ax_twinx)

    # get the t660 for the 2d case
    # pick the right time step
    slab_morph_path = os.path.join(local_dir_2d, "vtk_outputs", "slab_morph_t1.00e+06.txt")
    assert(os.path.isfile(slab_morph_path))

    data = np.loadtxt(slab_morph_path)
    steps = data[:, 1]
    times = data[:, 2]
    trenches = data[:, 3]
    slab_depths = data[:, 4]
    dip_100s = data[:, 5]

    # time of slab tip reaching 660 km and the index in the list
    sfunc = interp1d(slab_depths, times, assume_sorted=True)
    t660_2d = sfunc(660e3)
    i660_2d = IndexByValue(times, t660_2d)
    step660 = steps[i660_2d]
    print("2d case: t660 = %.4e Ma, step660 = %d" % (t660_2d/1e6, step660))

# options of the plot
end_time = 20e6 # yr
ax.set_xlim([0.0, end_time / 1e6])
ax.set_ylim([-300.0, 100.0])
ax.set_xlabel('Time (Myr)')
ax.set_ylabel('Trench position (km)')
ax_twinx.set_ylim([0.0, 1000.0])
ax_twinx.set_ylabel('Slab depth (km)')
ax.legend()
ax.grid()

############
# start the second plot: tip and dip angle
ax2 = fig.add_subplot(gs[1, 0])
ax2_twinx = ax2.twinx()
for i in range(len(w_queries)):
    w_query = w_queries[i]
    _color = colors[i]
    ts, depths, dips = GetSlabDipAngle(local_dir, time_interval_for_slab_morphology, axis=ax2,\
         axis_twinx=ax2_twinx, w_query=w_query, silence=False, color=_color, crust_only=1)
    '''debug
    print("ts:")
    print(ts)
    print("dephs:")
    print(depths)
    print("dips:")
    print(dips)
    '''

    # todo debug
    PlotSlabDipAngle(local_dir, time_interval_for_slab_morphology, axis=ax2,\
         axis_twinx=ax2_twinx, w_query=w_query, silence=False, color=_color, crust_only=1)

if local_dir_2d is not None:
    TwoDVtkPp.PlotSlabDip100km2dInter1Ma(SlabPlot2d, local_dir_2d, axis=ax2) 

# options of the plot
ax2.set_xlim([0.0, end_time / 1e6])
ax2.set_ylim([30.0, 80.0])
ax2.set_xlabel('Time (Myr)')
ax2.set_ylabel('Dip Angle 100 km (deg)')
ax2_twinx.set_xlim([0.0, end_time / 1e6])
ax2_twinx.set_ylim([0.0, 1000.0])
ax2_twinx.set_xlabel('Time (Myr)')
ax2_twinx.set_ylabel('Slab depth (km)')
ax2_twinx.legend()
ax2_twinx.grid()

fig.tight_layout()
fig_path = os.path.join(local_dir, 'img', 'slab_morphology_2d_combined.pdf')
fig.savefig(fig_path)
print("Saved figure: ", fig_path)

plt.close()

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
print(local_dir_2d)

### Automatized Visualization with Paraview

run the following command with the script generated in the last block:

    paraview --script {paraview scripts}

In [ ]:
# plot the trench velocities in episodes
time_interval = 1e6

SlabPlot =  ThDVtkPp.SLABPLOT('slab')
episodes = [[0, 6], [5, 11], [10, 30]]
SlabPlot.PlotTrenchPositionEpisodes(local_dir, episodes, time_interval=time_interval)

# plt.close()

#### Functionalities to extract the slab surface in a cross section

In [ ]:
# Here I don't assign an additional crust_only field, thus I am using both sp_lower and sp_upper composition
# TODO: omit the intermediate results to not interfere with the previous section
vtu_snapshot = 100 + 4 # This is the snapshot = step + adaptive refinement

print("python -m shilofue.ThDSubduction0.VtkPp cross_section_at_depth -i %s -vss %d --slab_envelop_interval_y %.4e --slab_envelop_interval_z %.4e -ssc %.4e"\
        % (local_dir, vtu_snapshot, slab_envelop_interval_y, slab_envelop_interval_z, slab_shallow_cutoff))

In [ ]:
# plot the cross section of the slab surface at a given depth

depth=150e3
slab_surface_file="/home/lochy/ASPECT_PROJECT/aspectLib/.test/test_ThDSubduction_VtkPp/test_extract_slab_cross_section_at_depth/vtk_outputs/slab_surface_00144_d150.00km.txt"
assert(os.path.isfile(slab_surface_file))
data = np.loadtxt(slab_surface_file)
xs = data[:, 0]
ys = data[:, 1]
zs = data[:, 2]

fig, ax = plt.subplots()
ax.plot(xs/1e3, ys/1e3, "*")
ax.set_xlabel("X (km)")
ax.set_ylabel("Y (km)")
ax.set_title("Depth = %.2f km" % (depth/1e3))
ax.axis("equal")

#### Combine results for one case

I take the steps from the previous block where the paraview script is generated.

In [ ]:
# define the resize by width function to work with image.resize
resize_by_width = lambda size, width: (int(width), int(1.0 * size[1] *  width / size[0]))

# todo_comb
img_dir = os.path.join(local_dir, "img")
pv_output_dir = os.path.join(img_dir, "pv_outputs")
morphology_dir = os.path.join(img_dir, "morphology")
assert(os.path.isdir(pv_output_dir))
ns_image_path = os.path.join(img_dir, "newton_solver_history.png")
assert(os.path.isfile(ns_image_path))

# new image
# first initiate a new image
# The work flow of the Image module includes
# first openingthe figure and then extend that by
# opening new figures and paste them on the first one.
# One additional operation is appending text on the figure, this requries two additional packages
# ImageFont and ImageDraw
new_image_path = os.path.join(local_dir, "img", "case_combined.pdf")
image_size = (2700, 10000) # width, height
# image_size = (2700, int(h_last)) # width, height, do this if you want the exact height
new_image = Image.new('RGB',image_size,(250,250,250))
h_last = 0.0
h_interval = 100

# 1. paste the output from the new solver
# The figure is resized by the width of the canvas
ns_image = Image.open(ns_image_path)
ns_image = ns_image.resize(resize_by_width(ns_image.size, new_image.size[0]))
new_image.paste(ns_image, (0, 0))
h_last += ns_image.size[1]

# 2. paste the slab morphology plot
morphology_trench_image_path = os.path.join(morphology_dir, "trench_history.png")
morphology_velocity_image_path = os.path.join(morphology_dir, "trench_velocities.png")
assert(os.path.isfile(morphology_trench_image_path))
assert(os.path.isfile(morphology_velocity_image_path))
morphology_trench_image = Image.open(morphology_trench_image_path)
morphology_velocity_image = Image.open(morphology_velocity_image_path)
morphology_trench_image = morphology_trench_image.resize(resize_by_width(morphology_trench_image.size, new_image.size[0]/2.0))
morphology_velocity_image = morphology_velocity_image.resize(resize_by_width(morphology_velocity_image.size, new_image.size[0]/2.0))
new_image.paste(morphology_trench_image, (0, int(h_last+h_interval)))
new_image.paste(morphology_velocity_image, (int(new_image.size[0]/2.0), int(h_last+h_interval)))
h_last += max(morphology_trench_image.size[1], morphology_velocity_image.size[1])

# 3. paste the outputs from paraview
# Note "Visit_Options" is from a previous section where these plots are generated
steps = Visit_Options.options['GRAPHICAL_STEPS']
steps = [0, 130]
print("steps: ", steps)
fnt0 = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", int(h_interval / 3.0 * 2.0))
for step in steps:
    _time = step * 0.1e6
    print(_time) # debug
    # a. plot of viscosity 
    vis_center_image_path = os.path.join(pv_output_dir, "slice_trench_center_y_viscosity_t%.4e.png" % _time)
    assert(os.path.isfile(vis_center_image_path))
    vis_center_image = Image.open(vis_center_image_path)
    # print(vis_image.size)
    vis_center_image = vis_center_image.resize(resize_by_width(vis_center_image.size, new_image.size[0]/2.0))
    new_image.paste(vis_center_image, (0, int(h_last) + h_interval))
    # b. plot of the whole mantle
    vis_edge_image_path = os.path.join(pv_output_dir, "slice_trench_edge_y_viscosity_t%.4e.png" % _time)
    assert(os.path.isfile(vis_edge_image_path))
    vis_edge_image = Image.open(vis_edge_image_path)
    vis_edge_image = vis_edge_image.resize(resize_by_width(vis_edge_image.size, new_image.size[0]/2.0))
    new_image.paste(vis_edge_image, (int(np.ceil(new_image.size[0]/2.0)), int(h_last) + h_interval))
    h_last1 = h_last + vis_edge_image.size[1] + 0.5 * h_interval
    # c. cross section along z direction and plot the flow filed
    flow_100_image_path = os.path.join(pv_output_dir, "slice_100.0km_t%.4e.png" % _time)
    flow_200_image_path = os.path.join(pv_output_dir, "slice_200.0km_t%.4e.png" % _time)
    flow_100_image =  Image.open(flow_100_image_path)
    new_image.paste(flow_100_image, (0, int(h_last1)))
    flow_200_image =  Image.open(flow_200_image_path)
    new_image.paste(flow_200_image, (int(np.ceil(new_image.size[0]/2.0)), int(h_last1)))
    # d. plot a sub-title, including the time
    d = ImageDraw.Draw(new_image)
    d.text((int(new_image.size[0]/2.0), h_last), "t = %.2f Ma" % (_time/1e6), font=fnt0, fill=(0, 0, 0))
    h_last = h_last1 + flow_100_image.size[1] + h_interval

# save to a new figure
new_image.save(new_image_path)

### Pick Timesteps

Pick in WebPlotDigitizer the time steps of:

1. Slab tip reaching 660 ($t_{660}$).
2. First peak of trench motion after $t_{660}$ ($t_{p1}$), and trench velocity
3. Second peak of trench motion after $t_{660}$ ($t_{p2}$), and trench velocity

### Visualization, pt 2 : save plots at these steps

Navigate to the paraview_scripts/slab.py file and change the steps.

### Plot Morphology Extra : Compare Morphology

Compare the slab morphology from multiple cases.

In [ ]:
case_name1 = "EBA_CDPT9/eba_cdpt_SA80.0_OA40.0_pc_mei_gr10_cf25GPa_PT1800_1"

json_option =\
{
    "_comment" : "This is configuration for combining results of time run (time, step, dofs ...)",
    "case_root": local_TwoDSubduction_dir,
    "cases": [case_name, case_name1],
    "output directory": {
        "relative": 1,
        "path": "plot_combine"
    },
    "width": -1.0,
    "time range": [0.0, 60e6],
    "trench position range": [-200e3, 200e3],
    "slab depth range": [0e3, 1800e3]
}

# print("json_option: ", json_option) # debug
PlotCombineExecute(TwoDVtkPp.PLOT_COMBINE_SLAB_MORPH, TwoDVtkPp.PC_MORPH_OPT, "slab_morph", json_option)

#### Make Animation

First generate the script for visualizations.
The time interval of visualizations for animation could be set through

    time_interval = 0.5e6

In [ ]:
# todo_ani

# turn on plot_axis if I want to save a complete result
# turn off if I want to prepare for figures in a paper
plot_axis = True
time_interval = 1e6
max_velocity = -1.0  # rescale the color for velocity

ThDPlotCase.PlotCaseRun(local_dir, time_range=None, run_visual=False,\
        time_interval=time_interval, visualization="paraview", step=None, plot_axis=plot_axis, max_velocity=-1.0)
plt.close() # plot won't show up below

_src = os.path.join(ASPECT_LAB_DIR, "files/ThDSubduction/bash_scripts/make_animation_paraview.sh")
_dst = os.path.join(local_dir, "paraview_scripts")
assert(os.path.isfile(_src))
shutil.copy(_src, _dst)

# replot the results of the morphology
# ThDPlotCase.PlotMorphAnimeCombined(local_dir, time_interval=time_interval)
# plt.close()

In [ ]:
# TODO: fix the flots and combine that into animation

SlabPlot = ThDVtkPp.SLABPLOT('slab')
vtu_step_list=[0, 15, 28]
this_vtu_step=200
SlabPlot.PlotTrenchPositionAnimation(local_dir, vtu_step_list, this_vtu_step, time_interval=1e6)

Then run scripts with

    pvpython slab.py

Note this will generate the visualizations for all the steps

Next, make the animation.

Assign the duration for each frame:

    duration = 0.2

By default, we remove the previous results and generate new results

    remove_old_results = True

Two default files are imbeded with operations for chunk and box geometry.
These operations will be performed on each figure to make the final animation.

In [ ]:
# this two are the json files for the order of options to do in imageio
# following the options defined in this two files, the results would be a combination of result for
# one single computation step.
time_interval = 1e6
duration = 0.2 # time for each frame
remove_old_results = True

default_box = os.path.join(ASPECT_LAB_DIR, "files", "ThDSubduction", "figure_step_template_box_04262024.json")

# remove old results
if remove_old_results:
    temp_dir = os.path.join(local_dir, "img", "pv_outputs", "temp")
    if os.path.isdir(temp_dir):
        rmtree(temp_dir)

pr_script = ThDPlotCase.PrScriptToUse(local_dir, default_box)
Plotter = ThDPlotCase.PLOTTER(PREPARE_RESULT_OPTIONS, [ThDPlotCase.PlotCaseRun]) # note we don't want to replot things here
PlotCase.AnimateCaseResults(Plotter.PlotPrepareResultStep, local_dir, pr_script, time_interval=time_interval, duration=duration)